In [2]:
import numpy as np
import pandas as pd
import ee
import folium
from folium import plugins
import json
import geopandas as gpd
import matplotlib as mpl
import fiona
import datetime
from csv import writer

In [3]:
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [4]:
def GetImgFromSat(GPDshp , date_from, date_to, SatCode = 'COPERNICUS/S2_SR'):
    Geojs = json.loads(GPDshp.to_json())

    geometry = ee.Geometry(ee.FeatureCollection(Geojs).geometry())

    sat = ee.ImageCollection(SatCode)

    img = sat.filterBounds(geometry).filterDate(date_from, date_to)
    
    return img

In [25]:
# computation

def manualNDWI(img, b3 = 'B3', b8 = 'B8'):
    try:
        return img.select(b3).subtract(img.select(b8)).divide(img.select(b3).add(img.select(b3)))
    except:
        return img 

def manualFAPAR(img, b4 = 'B4', b8 = 'B8'):
    try:
        return img.select(b8).subtract(img.select(b4)).divide(img.select(b8).add(img.select(b4)))
    except:
        return img 
def manualNDVI(img, b5 = 'B5', b4 = 'B4'):
    try:
        return img.select(b5).subtract(img.select(b4)).divide(img.select(b5).add(img.select(b4)))
    except:
        return img 
        
def manualStag(NDWI_img, FAPAR_img, threshold=0):
  return NDWI_img.multiply(FAPAR_img).gt(threshold)

def manualNDMI(img, b5 = 'B5', b6 = 'B6'):
  return img.select(b5).subtract(img.select(b6)).divide(img.select(b5).add(img.select(b6)))

def manualAerosol(img, aerosol = 'SR_QA_AEROSOL'):
  return img. select(aerosol).rename('aerosol')

def manualSurfTemp(img, SurfTemp = 'LST_Day_1km'):
  return img.select(SurfTemp).rename('surface_temparature')

In [26]:
def ShpToGeometry(shp):
  feature = ShpToEEFeature(shp)
  geometry = ee.Geometry(feature.geometry())
  return geometry

def ShpToEEFeature(shp):
  js = json.loads(shp.to_json())
  feature = ee.FeatureCollection(js)
  return feature

In [27]:
locations = {
    'QC': 'QC.shp',
    'Dumaguete': 'Dumaguete.shp',
    'Iloilo': 'Iloilo.shp',
    'Calabarzon': 'CALABARZON.shp',
    'Tacloban': 'Tacloban.shp',
    'Cotabato': 'Cotabato.shp',
}

# layers = {
#     'ndvi': '',
#     'ndwi': '',
#     'fapar': ''
# }

In [32]:
start_date = "2019-01-01"
end_date = "2022-08-30"

In [33]:
for location, shapefile in locations.items():
    load_shp = gpd.read_file("/work/Priority Shape Files/" + shapefile)
    print(location)

    start = datetime.datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.datetime.strptime(end_date, '%Y-%m-%d')

    days = abs(start-end).days
    total_weeks = days//7

 
    for week in range(total_weeks):
        current_date = (start.date() + datetime.timedelta(days = 7 * week)).isoformat()
        current_end_date = (start.date() + datetime.timedelta(days = 7 * week) + datetime.timedelta(days = 7)).isoformat()

        try:
                        
            img = GetImgFromSat(load_shp, current_date, current_end_date)
        
            # compute index
            ndwi = manualNDWI(img.median())
            fapar = manualFAPAR(img.median())
            ndvi=manualNDVI(img.median())

            # get mean values
            avg_ndwi = ndwi.reduceRegion(
                        reducer=ee.Reducer.mean(),
                        geometry=ShpToGeometry(load_shp),
                        scale=30,
                        bestEffort=True
                    ).getInfo()
            avg_fapar = fapar.reduceRegion(
                        reducer=ee.Reducer.mean(),
                        geometry=ShpToGeometry(load_shp),
                        scale=30,
                        bestEffort=True
                    ).getInfo()
            avg_ndvi = ndvi.reduceRegion(
                        reducer=ee.Reducer.mean(),
                        geometry=ShpToGeometry(load_shp),
                        scale=30,
                        bestEffort=True
                    ).getInfo()

            avg_ndwi = list(avg_ndwi.values())[0]
            avg_fapar = list(avg_fapar.values())[0]
            avg_ndvi = list(avg_ndvi.values())[0]

            res = [location, avg_ndwi, avg_fapar, avg_ndvi, current_date, current_end_date]
        except Exception as e: 
            print(e)
            res = [location, None, None, None, current_date, current_end_date]
            
        with open('ee_location_indices.csv', 'a') as f_object:
            writer_object = writer(f_object)
            writer_object.writerow(res)
            f_object.close()
        print(res)

QC


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0cd7ca08-c036-4633-8a95-c1f919155399' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>